In [2]:
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task, before_kickoff, after_kickoff
from crewai_tools import PDFSearchTool


In [3]:
from crewai_tools import PDFSearchTool, DirectoryReadTool
# Uncomment the following line to use an example of a custom tool
# from projeto_1.tools.custom_tool import MyCustomTool
# Uncomment the following line to use an example of a knowledge source
# from crewai.knowledge.source.text_file_knowledge_source import TextFileKnowledgeSource

# Check our tools documentations for more information on how to use them
# from crewai_tools import SerperDevTool

directory_path="../../../../Papers/"
directory_read_tool=DirectoryReadTool(directory_path)
directory_read_tool

DirectoryReadTool(name='List files in directory', description="Tool Name: List files in directory\nTool Arguments: {}\nTool Description: A tool that can be used to list ../../../../Papers/'s content.", args_schema=<class 'crewai_tools.tools.directory_read_tool.directory_read_tool.FixedDirectoryReadToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x132027c40>, result_as_answer=False, directory='../../../../Papers/')

In [5]:
from crewai_tools import ScrapeWebsiteTool

# To enable scrapping any website it finds during it's execution
tool = ScrapeWebsiteTool()

# Initialize the tool with the website URL, 
# so the agent can only scrap the content of the specified website
tool = ScrapeWebsiteTool(website_url='https://playground.powerbi.com/en-us/showcases-gallery/personalize-top-insights')

# Extract the text from the site
text = tool.run()
print(text)

Using Tool: Read website content
Power BI Playground


In [17]:
from crewai_tools import VisionTool

import os

os.environ["OPENAI_API_KEY"]="put the key"

# To enable scrapping any website it finds during it's execution
tool = VisionTool()

# Initialize the tool with the website URL, 
# so the agent can only scrap the content of the specified website
tool = VisionTool(image_path_url='https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Sunflower_sky_backdrop.jpg/1280px-Sunflower_sky_backdrop.jpg')


In [18]:
# Extract the text from the site
text = tool.run()
print(text)

Using Tool: Vision Tool
Image Path or URL is required.


In [16]:
type(text)

str

In [5]:
pdf_reader=PDFSearchTool()

In [7]:
agent_1 = Agent(
  role='Research Analyst',
  goal='Provide a summary of the given articles',
  backstory='An expert analyst with a keen eye for market trends.',
  tools=[pdf_reader],
  memory=True, # Enable memory
  verbose=True,
  #max_rpm=None, # No limit on requests per minute
  #max_iter=25, # Default value for maximum iterations
)

In [8]:
task_1 = Task(
    description="Review the context you got and expand each topic into a full section for a report."
    "Make sure the report is detailed and contains any and all relevant information.",
  expected_output="A fully fledge reports with the mains topics, each with a full section of information."
    "Formatted as markdown without '```'",
  agent= agent_1
)

In [ ]:
import sys
from crewai import Agent, Task
import os
from dotenv import load_dotenv
from crewai import Crew, Process
from langchain_openai import AzureChatOpenAI

load_dotenv()

default_llm = AzureChatOpenAI(
    openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt35"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://<your-endpoint>.openai.azure.com/"),
    api_key=os.environ.get("AZURE_OPENAI_KEY")
)


# Create a researcher agent
researcher = Agent(
  role='Senior Researcher',
  goal='Discover groundbreaking technologies',
  verbose=True,
  llm=default_llm,
  backstory='A curious mind fascinated by cutting-edge innovation and the potential to change the world, you know everything about tech.'
)

# Task for the researcher
research_task = Task(
  description='Identify the next big trend in AI',
  agent=researcher  # Assigning the task to the researcher
)


# Instantiate your crew
tech_crew = Crew(
  agents=[researcher],
  tasks=[research_task],
  process=Process.sequential  # Tasks will be executed one after the other
)

# Begin the task execution
tech_crew.kickoff()